# 예측모델 전처리

In [1]:
# 모듈 Importing 및 경고 끄기
import warnings
warnings.filterwarnings(action='ignore')

import random
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
# 데이터 불러오기
df_data = pd.read_csv('./train_female.csv')
df_predict = pd.read_csv('./test_female.csv')
df_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_predict.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
# 불러온 데이터 확인
df_data

,결과,득점_득점,공격종합성공률,후위성공률,퀵오픈성공률,서브성공률,디그성공률,세트성공률,리시브성공률,블로킹성공률,...,팀명_한국도로공사,팀명_현대건설,팀명_흥국생명,상대팀_GS칼텍스,상대팀_IBK기업은행,상대팀_KGC인삼공사,상대팀_페퍼저축은행,상대팀_한국도로공사,상대팀_현대건설,상대팀_흥국생명
0,1,88,0.413793,0.444444,0.423077,0.082569,0.839286,0.374302,0.443299,0.097222,...,0,0,0,0,0,0,0,1,0,0
1,0,83,0.422360,0.592593,0.551020,0.096154,0.789474,0.402439,0.421053,0.084746,...,0,0,0,0,0,1,0,0,0,0
2,0,39,0.358696,0.450000,0.300000,0.046512,0.811321,0.324074,0.666667,0.181818,...,0,0,0,0,0,0,0,0,1,0
3,0,47,0.401961,0.533333,0.517241,0.078431,0.881356,0.405941,0.389831,0.058824,...,0,0,0,0,1,0,0,0,0,0
4,1,85,0.458065,0.400000,0.619048,0.018519,0.816327,0.421384,0.467391,0.181818,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,0,37,0.357143,0.250000,0.423077,0.022222,0.756098,0.190476,0.267857,0.157895,...,0,0,1,1,0,0,0,0,0,0
944,0,58,0.316770,0.142857,0.400000,0.054795,0.839286,0.311258,0.246753,0.058824,...,0,0,1,0,0,0,0,1,0,0
945,1,62,0.413793,0.357143,0.520000,0.082192,0.804878,0.360360,0.392857,0.163265,...,0,0,1,0,0,1,0,0,0,0
946,0,60,0.327044,0.153846,0.379310,0.037037,0.862385,0.308642,0.382716,0.062500,...,0,0,1,0,0,0,1,0,0,0


In [4]:
# 각팀 별 데이터 DF 생성
GS칼텍스 = df_data[df_data['팀명_GS칼텍스'] == 1]
GS칼텍스 = GS칼텍스.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_GS칼텍스'])

IBK기업은행 = df_data[df_data['팀명_IBK기업은행'] == 1]
IBK기업은행 = IBK기업은행.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_IBK기업은행'])

KGC인삼공사 = df_data[df_data['팀명_KGC인삼공사'] == 1]
KGC인삼공사 = KGC인삼공사.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_KGC인삼공사'])

페퍼저축은행 = df_data[df_data['팀명_페퍼저축은행'] == 1]
페퍼저축은행 = 페퍼저축은행.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_페퍼저축은행'])

한국도로공사 = df_data[df_data['팀명_한국도로공사'] == 1]
한국도로공사 = 한국도로공사.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_한국도로공사'])

현대건설 = df_data[df_data['팀명_현대건설'] == 1]
현대건설 = 현대건설.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_현대건설'])

흥국생명 = df_data[df_data['팀명_흥국생명'] == 1]
흥국생명 = 흥국생명.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_흥국생명'])

In [5]:
# 각 팀별 예측대상 데이터 DF 생성
GS칼텍스_predict = df_predict[df_predict['팀명_GS칼텍스'] == 1]
GS칼텍스_predict = GS칼텍스_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_GS칼텍스'])
GS칼텍스_predict =GS칼텍스_predict.reset_index().drop(columns='index')

IBK기업은행_predict = df_predict[df_predict['팀명_IBK기업은행'] == 1]
IBK기업은행_predict = IBK기업은행_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_IBK기업은행'])
IBK기업은행_predict =IBK기업은행_predict.reset_index().drop(columns='index')

KGC인삼공사_predict = df_predict[df_predict['팀명_KGC인삼공사'] == 1]
KGC인삼공사_predict = KGC인삼공사_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_KGC인삼공사'])
KGC인삼공사_predict =KGC인삼공사_predict.reset_index().drop(columns='index')

페퍼저축은행_predict = df_predict[df_predict['팀명_페퍼저축은행'] == 1]
페퍼저축은행_predict = 페퍼저축은행_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_페퍼저축은행'])
페퍼저축은행_predict =페퍼저축은행_predict.reset_index().drop(columns='index')

한국도로공사_predict = df_predict[df_predict['팀명_한국도로공사'] == 1]
한국도로공사_predict = 한국도로공사_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_한국도로공사'])
한국도로공사_predict =한국도로공사_predict.reset_index().drop(columns='index')

현대건설_predict = df_predict[df_predict['팀명_현대건설'] == 1]
현대건설_predict = 현대건설_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_현대건설'])
현대건설_predict =현대건설_predict.reset_index().drop(columns='index')

흥국생명_predict = df_predict[df_predict['팀명_흥국생명'] == 1]
흥국생명_predict = 흥국생명_predict.drop(columns=['팀명_GS칼텍스', '팀명_IBK기업은행', '팀명_KGC인삼공사', '팀명_페퍼저축은행',
       '팀명_한국도로공사', '팀명_현대건설', '팀명_흥국생명', '상대팀_흥국생명'])
흥국생명_predict =흥국생명_predict.reset_index().drop(columns='index')

# 각 팀 별 모델 만들기

팀별 학습 진행후, 제일 좋은 모델을 기반으로 팀, 상대팀 입력  
팀의 스탯은 평균값을 중심으로 min~max 범위내로 조정해서 함수 입력  
상대팀과의 상성과 임의로 주어진 스탯을 중심으로 승패 계산  

In [6]:
# 분류 모델 생성 및 하이퍼 파라미터 지정
lgb = LGBMClassifier(bagging_fraction=0.9, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, feature_fraction=0.5,
                importance_type='split', learning_rate=0.4, max_depth=-1,
                min_child_samples=6, min_child_weight=0.001, min_split_gain=0.3,
                n_estimators=20, n_jobs=-1, num_leaves=150, objective=None,
                random_state=0, reg_alpha=0.005, reg_lambda=0.0005,
                silent='warn', subsample_for_bin=200000)
rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                        class_weight='balanced_subsample', criterion='entropy',
                        max_depth=4, max_features='log2', max_leaf_nodes=None,
                        max_samples=None, min_impurity_decrease=0.0002,
                        min_samples_leaf=5,
                        min_samples_split=9, min_weight_fraction_leaf=0.0,
                        n_estimators=130, n_jobs=-1, oob_score=False,
                        random_state=0, verbose=0, warm_start=False)
gbc = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.01, loss='deviance', max_depth=7,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.05,
                            min_samples_leaf=2, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=140,
                            n_iter_no_change=None,
                            random_state=0, subsample=0.35, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False)


In [7]:
# 각 팀별 모델 피팅
# GS칼텍스
model_GS칼텍스 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= GS칼텍스.drop(columns=['결과'])
y = GS칼텍스['결과']
model_GS칼텍스.fit(X,y)

#IBK기업은행
model_IBK기업은행 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= IBK기업은행.drop(columns=['결과'])
y = IBK기업은행['결과']
model_IBK기업은행.fit(X,y)

#KGC인삼공사
model_KGC인삼공사 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= KGC인삼공사.drop(columns=['결과'])
y = KGC인삼공사['결과']
model_KGC인삼공사.fit(X,y)

#페퍼저축은행
model_페퍼저축은행 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 페퍼저축은행.drop(columns=['결과'])
y = 페퍼저축은행['결과']
model_페퍼저축은행.fit(X,y)

#한국도로공사
model_한국도로공사 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 한국도로공사.drop(columns=['결과'])
y = 한국도로공사['결과']
model_한국도로공사.fit(X,y)

#현대건설
model_현대건설 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 현대건설.drop(columns=['결과'])
y = 현대건설['결과']
model_현대건설.fit(X,y)

#흥국생명
model_흥국생명 = VotingClassifier([('lgb', lgb), ('rf',rf), ('gbc',gbc)], voting= 'soft')
X= 흥국생명.drop(columns=['결과'])
y = 흥국생명['결과']
model_흥국생명.fit(X,y)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


VotingClassifier(estimators=[('lgb',
                              LGBMClassifier(bagging_fraction=0.9,
                                             bagging_freq=3,
                                             feature_fraction=0.5,
                                             learning_rate=0.4,
                                             min_child_samples=6,
                                             min_split_gain=0.3,
                                             n_estimators=20, num_leaves=150,
                                             random_state=0, reg_alpha=0.005,
                                             reg_lambda=0.0005)),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced_subsample',
                                                     criterion='entropy',
                                                     max_depth=4,
                                                     max_features='log2',
             

In [8]:
# 랜덤 스탯 부여 함수
def pre_predict(x,x_predict):
    for i in range(len(x_predict)):
        if x_predict.iloc[i,10] == 1:
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,10]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,10]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,10]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,10]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,10]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,10]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,10]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,10]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,10]==1]['블로킹성공률'].values)      
        if x_predict.iloc[i,11] == 1:
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,11]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,11]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,11]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,11]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,11]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,11]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,11]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,11]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,11]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,12] == 1: 
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,12]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,12]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,12]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,12]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,12]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,12]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,12]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,12]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,12]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,13] == 1:  
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,13]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,13]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,13]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,13]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,13]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,13]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,13]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,13]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,13]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,14] == 1:       
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,14]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,14]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,14]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,14]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,14]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,14]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,14]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,14]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,14]==1]['블로킹성공률'].values)
        if x_predict.iloc[i,15] == 1:            
            x_predict.loc[i,'득점_득점'] = random.choice(x[x.iloc[:,15]==1]['득점_득점'].values)
            x_predict.loc[i,'공격종합성공률'] = random.choice(x[x.iloc[:,15]==1]['공격종합성공률'].values)
            x_predict.loc[i,'후위성공률'] = random.choice(x[x.iloc[:,15]==1]['후위성공률'].values)
            x_predict.loc[i,'퀵오픈성공률'] = random.choice(x[x.iloc[:,15]==1]['퀵오픈성공률'].values)
            x_predict.loc[i,'서브성공률'] = random.choice(x[x.iloc[:,15]==1]['서브성공률'].values)
            x_predict.loc[i,'디그성공률'] = random.choice(x[x.iloc[:,15]==1]['디그성공률'].values)
            x_predict.loc[i,'세트성공률'] = random.choice(x[x.iloc[:,15]==1]['세트성공률'].values)
            x_predict.loc[i,'리시브성공률'] = random.choice(x[x.iloc[:,15]==1]['리시브성공률'].values)
            x_predict.loc[i,'블로킹성공률'] = random.choice(x[x.iloc[:,15]==1]['블로킹성공률'].values)
            
    predict_data = pd.DataFrame(x_predict.drop(columns='결과'))
    return predict_data

In [9]:
# 승패 예측 함수
def winning_predict(model,x):
    win_lose = model.predict(x)
    for i in range(len(win_lose)):
        if win_lose[i] == 1:
            if x.iloc[i,9] == 1:
                print(f'{x.columns[9]}에게 승리')
            elif x.iloc[i,10] == 1:
                print(f'{x.columns[10]}에게 승리')            
            elif x.iloc[i,11] == 1:
                print(f'{x.columns[11]}에게 승리')
            elif x.iloc[i,12] == 1:
                print(f'{x.columns[12]}에게 승리')
            elif x.iloc[i,13] == 1:
                    print(f'{x.columns[13]}에게 승리')    
            elif x.iloc[i,14] == 1:
                    print(f'{x.columns[14]}에게 승리')    
        else:
            if x.iloc[i,9] == 1:
                print(f'{x.columns[9]}에게 패배')
            elif x.iloc[i,10] == 1:
                print(f'{x.columns[10]}에게 패배')            
            elif x.iloc[i,11] == 1:
                print(f'{x.columns[11]}에게 패배')
            elif x.iloc[i,12] == 1:
                print(f'{x.columns[12]}에게 패배')
            elif x.iloc[i,13] == 1:
                print(f'{x.columns[13]}에게 패배')    
            elif x.iloc[i,14] == 1:
                print(f'{x.columns[14]}에게 패배')       

In [15]:
# 입출력 구현
team_model_list =  [model_GS칼텍스,model_IBK기업은행,model_KGC인삼공사,model_페퍼저축은행,model_한국도로공사,model_현대건설,model_흥국생명]
team_name_list =  'model_GS칼텍스,model_IBK기업은행,model_KGC인삼공사,model_페퍼저축은행,model_한국도로공사,model_현대건설,model_흥국생명'.split(',')
predict_list = [GS칼텍스_predict,IBK기업은행_predict,KGC인삼공사_predict,페퍼저축은행_predict,한국도로공사_predict,현대건설_predict,흥국생명_predict]
team_list = [GS칼텍스,IBK기업은행,KGC인삼공사,페퍼저축은행,한국도로공사,현대건설,흥국생명]

while True:
    team_name = input(f'팀 이름을 입력하세요(GS칼텍스, IBK기업은행, KGC인삼공사, 페퍼저축은행, 한국도로공사, 현대건설, 흥국생명): ')
    team_name_save = team_name
    team_name = 'model_'+team_name
    predict_name = team_name + '_predict'
    print(team_name)
    if team_name in team_name_list:
        for i in range(len(team_name_list)):
            if team_name == team_name_list[i]:
                team_name = team_model_list[i]
                predict_name = predict_list[i]
                team_name_save = team_list[i]
                break
        x = pre_predict(team_name_save,predict_name)
        winning_predict(team_name, x)
        pred = team_name.predict(x)      
        rate = round(len(predict_name[predict_name['결과'] == pred]) / len(predict_name)*100,2)
        print(f'적중률 : {rate}%')
        break
    else:
        continue

팀 이름을 입력하세요(GS칼텍스, IBK기업은행, KGC인삼공사, 페퍼저축은행, 한국도로공사, 현대건설, 흥국생명): 흥국생명
model_흥국생명
상대팀_IBK기업은행에게 패배
상대팀_한국도로공사에게 패배
상대팀_GS칼텍스에게 승리
